In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the data from the UN FAO DAD-IS DB
# Domestic Animal Diversity Information System (DAD-IS)
dad_is_df = pd.read_csv('../data/010522_FAO_DAD-IS_export.csv')
dad_is_df = dad_is_df.replace({'Duck (domestic)': 'Duck', 'Duck(domestic) Muscovy duck': 'Muscovy Duck',
                               'Goose (domestic)': 'Goose', 'Dromedary Bactrian Camel': 'Bactrian Camel',
                               'Vicuña': 'Vicuna', 'Yak (domestic)': 'Yak'})

cols_of_interest = ['Weight males', 'Weight females', 'Birth weight males', 'Birth weight females',
                    'Age maturity males', 'Age maturity females', 'Age breeding males', 'Age breeding females',
                    'Age first parturition (month) AVG', 'Age first parturition (month) MIN', 'Age first parturition (month) MAX',
                    'Parturition interval (day) AVG', 'Parturition interval (day) MIN', 'Parturition interval (day) MAX',
                    'Length productive life (years)', 'Daily gain', 'Carcass weight (kg)', 'Dressing percentage (%)',
                    'Litter size AVG', 'Litter size MIN', 'Litter size MAX']

# Renaming map for the columns to include units where absent.
# Units gleaned from GUI interface here:
# https://www.fao.org/dad-is/browse-by-country-and-species/en/
colname_map = dict((k,k) for k in cols_of_interest)
kg_cols = ['Weight males', 'Weight females', 'Birth weight males', 'Birth weight females']
for c in kg_cols:
    colname_map[c] = c + ' (kg)'
month_cols = ['Age maturity males', 'Age maturity females', 'Age breeding males', 'Age breeding females']
for c in month_cols:
    colname_map[c] = c + ' (month)'
colname_map['Daily gain'] = 'Daily gain (g)'

# Zero values are non-sensical for all the columns - should be NaN for log-scale plotting
for c in cols_of_interest:
    mask = dad_is_df[c] == 0
    dad_is_df.loc[mask, c] = np.nan
    
# Drop everything that is missing data in all columns of interest. 
dad_is_df = dad_is_df.dropna(axis=0, how='all', subset=cols_of_interest)

#dad_is_df.columns.tolist()

/Users/flamholz/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dad_is_df.columns.tolist()

['Country',
 'ISO3',
 'Specie',
 'Breed/Most common name',
 'Language',
 'Description',
 'Transboundary name',
 'Other name',
 'Provision Uses',
 'Regulation and Maintenance Uses',
 'Cultural Uses',
 'Description Of Specific Uses',
 'Production environment/management systems',
 'Literature related to uses and ecosystem services',
 'Additional information',
 'Additional information comments',
 'Efabis cultural role comment',
 'Efabis cultural value',
 'Adaptability to specific environment',
 'Specific resistance or tolerance',
 'Specific reproductive characteristic',
 'Special characteristic of product',
 'Other special qualities',
 'Reference for special qualities',
 'Efabis genetic features',
 'Efabis environmental role',
 'Efabis adaptability to marginal land',
 'Body conformation',
 'Coat description',
 'Coat quality',
 'Comb type',
 'Skin colour',
 'Shank and foot colour',
 'Plumage colour',
 'Pattern within feather',
 'Avian classification',
 'Color comments',
 'Efabis main colour

The next few cells are printing out some outlier data to document some clear errors in DAD-IS. 

In [4]:
# Look at some of the absurd litter sizes in the dataset - several > 100!
# Hiding the pigs since commercial pigs can have litters > 10. 
mask = np.logical_and(dad_is_df['Litter size AVG'] > 10,
                      dad_is_df.Specie != 'Pig')
cols =  ['Specie', 'Breed/Most common name', 'Parturition interval (day) MIN',
         'Parturition interval (day) AVG', 'Parturition interval (day) MAX',
         'Litter size AVG', 'Litter size MIN', 'Litter size MAX', 
         'Weight males', 'Birth weight males',
         'Weight females', 'Birth weight females']
dad_is_df.loc[mask, cols]

,Specie,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Weight males,Birth weight males,Weight females,Birth weight females
6805,Goat,Boer goat,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN
9662,Goat,Macedonian Goat,NaN,NaN,NaN,15.0,NaN,NaN,35.0,2.50,45.0,2.20
10912,Goat,Aljabal Alakhdar,NaN,149.0,NaN,128.0,125.0,130.0,36.0,3.34,36.0,3.05
10916,Goat,Jabbali,148.0,152.0,156.0,126.0,120.0,133.0,36.0,3.30,31.0,2.90
13560,Sheep,Barbarine,NaN,NaN,NaN,112.0,NaN,NaN,NaN,3.50,NaN,3.20
13562,Sheep,Noire de Thibar,NaN,NaN,NaN,130.0,130.0,160.0,NaN,4.32,NaN,NaN
13564,Sheep,Sicilo-Sarde,NaN,NaN,NaN,144.0,NaN,NaN,70.0,3.40,45.0,3.20


In [5]:
# Look at some absurd birth weights that well-exceed the listed adult weights.
mask = np.logical_or(dad_is_df['Birth weight males'] > 0.9*dad_is_df['Weight males'],
                     dad_is_df['Birth weight females'] > 0.9*dad_is_df['Weight females'])

dad_is_df.loc[mask, cols]

,Specie,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Weight males,Birth weight males,Weight females,Birth weight females
1398,Sheep,Privorska,NaN,NaN,NaN,1.0,NaN,NaN,52.0,46.00,40.0,44.00
2399,Cattle,Tibetan,NaN,NaN,NaN,NaN,NaN,NaN,215.0,215.80,197.0,197.70
7322,Rabbit,Rexsi Agrinak,14.0,14.0,16.0,NaN,7.0,8.0,2.9,49.94,2.7,55.27
8894,Chicken,ALARABY CHICKENS,NaN,NaN,NaN,NaN,NaN,NaN,1.9,1.90,1.5,1.50
11279,Chicken,Banaba,NaN,NaN,NaN,NaN,NaN,NaN,1.6,29.86,1.5,29.86
13394,Chicken,Khiao Hu0ai Sai chicken,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.50,2.5,2.50
14032,Rabbit,Californian,NaN,NaN,NaN,NaN,8.0,10.0,4.4,450.00,4.4,NaN


In [7]:
# Look at some absurd partruition intervals (inter-birth intervals) that are too short to be reasonable. 
# ... From the website the parturition interval appears to be reported in days. 
# Even for a rabbit this is too short...
mask = dad_is_df['Parturition interval (day) AVG'] < 20 
masked_df = dad_is_df.loc[mask, cols]
for gidx, gdf in masked_df.groupby('Specie'):
    print(gidx)
    for idx, row in gdf.iterrows():
        print(row['Breed/Most common name'])
    
masked_df

Cattle
Blanc-Bleu type mixte
Bali
Donggala
Pesisir (sum )
Sumbawa
Puerto Rican
Winam
Chicken
Mongolian Local Hen
Goat
Criolla del Sur de Mendoza
Creole
Pig
Kalinga
Quezon
Rabbit
Rexsi Agrinak
Sheep
Black Head Somali
Spelsau
Namaqua Afrikaner


,Specie,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Weight males,Birth weight males,Weight females,Birth weight females
205,Goat,Criolla del Sur de Mendoza,14.0,12.00,10.0,1.30,1.0,3.0,79.6,2.78,46.79,2.54
892,Cattle,Blanc-Bleu type mixte,10.0,13.00,18.0,NaN,NaN,NaN,1100.0,46.00,700.00,42.00
5088,Sheep,Black Head Somali,NaN,10.46,NaN,NaN,NaN,NaN,29.5,NaN,25.80,NaN
6885,Goat,Creole,NaN,13.00,NaN,1.75,NaN,NaN,NaN,NaN,NaN,NaN
7149,Cattle,Bali,13.0,14.50,17.0,NaN,1.0,3.0,475.0,18.00,250.00,20.00
7153,Cattle,Donggala,15.0,18.00,24.0,NaN,1.0,1.0,266.0,NaN,266.00,NaN
7164,Cattle,Pesisir (sum ),330.0,12.00,390.0,NaN,NaN,NaN,200.0,13.00,150.00,11.00
7170,Cattle,Sumbawa,12.0,14.00,17.0,NaN,1.0,NaN,400.0,26.00,300.00,26.00
7322,Rabbit,Rexsi Agrinak,14.0,14.00,16.0,NaN,7.0,8.0,2.9,49.94,2.70,55.27
8314,Cattle,Puerto Rican,NaN,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As documented above, the DAD-IS dataset contains some outliers that are clearly errors. The cell below removes them.

In [ ]:
# Birth weights should not be remotely similar to adult weights.
mask = dad_is_df['Birth weight males'] > 0.9*dad_is_df['Weight males']
dad_is_df.loc[mask, 'Birth weight males'] = np.NaN
dad_is_df.loc[mask, 'Weight males'] = np.NaN

mask = dad_is_df['Birth weight females'] > 0.9*dad_is_df['Weight females']
dad_is_df.loc[mask, 'Birth weight females'] = np.NaN
dad_is_df.loc[mask, 'Weight females'] = np.NaN

# None of these animals has a litter size > 30 (being very conservative)
litter_cols = ['Litter size AVG', 'Litter size MIN', 'Litter size MAX']

# Species where litters are surely < 10 (being very conservative, see comment below).
species_l10 = 'Goat,Cattle,Sheep'.split(',')
for lc in litter_cols:
    mask = dad_is_df[lc] > 30
    dad_is_df.loc[mask, lc] = np.NaN
    
    mask = np.logical_and(dad_is_df[lc] > 10, dad_is_df.Specie.isin(species_l10))
    dad_is_df.loc[mask, lc] = np.NaN
    
# There are a few goat breeds marked as having litter sizes > 10, even > 100.
# These numbers are absurd and must be in error. 
# Data show that even quadruplets are rare, <2% of pregnancies and <3% of live young.
# These numbers do depend on the genotype, however, meaning that larger litters are selectable.
# See references on goat litter sizes: 
# 1. M. Peaker, Gestation period and litter size in the goat. Br. Vet. J. 134, 379–383 (1978).
# 2. M. Mellado, et al., Relationship between litter birthweight and litter size in five goat genotypes. Anim. Produc. Sci. 51, 490–490 (2011).
    
# None of these animals has an interbirth interval less than 20 days (being very conservative)
# Even rabbits have a gestation period > 20 days (≈30 days)
# See W. K. Wilson, F. J. Dudley, The duration of gestation in rabbit breeds and crosses. J. Genet. 50, 384–391 (1952).
interval_cols = ['Parturition interval (day) AVG', 'Parturition interval (day) MIN', 'Parturition interval (day) MAX']
for ic in interval_cols:
    mask = dad_is_df[ic] <= 20
    dad_is_df.loc[mask, ic] = np.NaN


The cell below calculates derived data for the remaining species and puts units in the column names for clarity. 

In [ ]:
# Calculate the number of young per year from the litter size and interval between litters.
litter_size = dad_is_df['Litter size AVG']

# Have to convert 0 interval into NaN otherwise we get infinite young per year
litter_interval = dad_is_df['Parturition interval (day) AVG']

young_per_year = litter_size / (litter_interval/365)
dad_is_df['Young per year AVG (number)'] = young_per_year

mean_adult_weight_kg = dad_is_df[['Weight males', 'Weight females']].mean(axis=1)
dad_is_df['Adult weight AVG (kg)'] = mean_adult_weight_kg
dad_is_df['Adult weight AVG (g)'] = mean_adult_weight_kg*1000

mean_birth_weight_kg = dad_is_df[['Birth weight males', 'Birth weight females']].mean(axis=1)
dad_is_df['Birth weight AVG (kg)'] = mean_birth_weight_kg
dad_is_df['Birth weight AVG (g)'] = mean_birth_weight_kg*1000

neonate_mass_kg = dad_is_df['Birth weight AVG (kg)']
young_per_year = dad_is_df['Young per year AVG (number)']
dad_is_df['Young mass per year AVG (kg)'] = neonate_mass_kg*young_per_year
dad_is_df['Young mass per year AVG (g)'] = neonate_mass_kg*young_per_year*1000

# Put units in the relevant column names
col_replacement_dict = dict((c, nc) for c, nc in zip(cols_of_interest, colname_map))
dad_is_clean_df = dad_is_df.rename(columns=col_replacement_dict)

# Save a new cleaner file with units for the relevant columns.
dad_is_clean_df.to_csv('../data/010522_FAO_DAD-IS_cleaned.csv')
dad_is_clean_df.shape

In [ ]:
# Save number of individual breeds for the relevant columns
counts_by_species = dad_is_clean_df.groupby('Specie')['Breed/Most common name'].count()
counts_by_species.name = 'breed_count'
counts_by_species.to_csv('../data/010522_FAO_DAD-IS_breed_counts_by_species.csv')

In [ ]:
# Print the species for which there are > 100 breed representatives
mask = counts_by_species > 100
species_enough_data = counts_by_species[mask].index.tolist()
species_enough_data

In [ ]:
# Save quantiles for the relevant columns for the species that have enough data.
new_colnames += ['Young per year AVG (number)', 'Adult weight AVG (kg)']
mask = dad_is_clean_df.Specie.isin(species_enough_data)
quantiles_by_species = dad_is_clean_df[mask].groupby('Specie').quantile([0.005, 0.05, 0.25, 0.5, 0.75, 0.95, 0.995])[new_colnames]
quantiles_by_species.index = quantiles_by_species.index.set_names(['Specie', 'quantile'])
quantiles_by_species.to_csv('../data/010522_FAO_DAD-IS_quantiles_by_species.csv')
quantiles_by_species.head()